# Arup University - Machine Learning & Artificial Intelligence
***
https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed
### 2019 - 2020 Individual assignment
# Road Traffic Accidents


# Simple Regression


__Gabor Jenei__
***

In [1]:
#Imports
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

south_west_penisula_higway = ['E06000028', 'E06000029', 'E10000009','E10000027', 'E06000026', 'E06000027', 'E10000008', 'E06000052']
south_west_penisula_localauth = [496, 580, 581, 582, 583, 584, 585, 586,587,588,589, 596,605,606,607,608,609,610,635,640,641,642,643,644,645,646,647]

In [2]:
df_accidents = pd.read_csv(r"C:\Projects\0_Training\AU_MLAI\IndividualProject\data\df_accidents.csv", low_memory=False, index_col=0)
df_casualties= pd.read_csv(r"C:\Projects\0_Training\AU_MLAI\IndividualProject\data\df_casualties.csv", low_memory=False, index_col=0)
#df_vehicles= pd.read_csv(r"C:\Projects\0_Training\AU_MLAI\IndividualProject\data\df_vehicles.csv", low_memory=False, index_col=0)

In [3]:
casaualty_severity = df_casualties.groupby(['Accident_Index','Casualty_Severity']).size().unstack(fill_value=0)
casaualty_severity.rename(columns={1: "Fatal", 2: "Serious", 3: "Slight"}, inplace=True)

accidents_s1 = pd.merge(df_accidents, casaualty_severity, on="Accident_Index")
#accidents_s1.rename({1: "Fatal", 2: "Serious", 3: "Slight"}, axis='columns')
accidents_s1.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location', 'Fatal', 'Serious', 'Slight'],
      dtype='object')

In [4]:
accidents_s1.head().transpose()

,0,1,2,3,4
Accident_Index,2018010080971,2018010080973,2018010080974,2018010080981,2018010080982
Location_Easting_OSGR,529150,542020,531720,541450,543580
Location_Northing_OSGR,182270,184290,182910,183220,176500
Longitude,-0.139737,0.046471,-0.102474,0.037828,0.065781
Latitude,51.5246,51.5397,51.5297,51.5302,51.4693
Police_Force,1,1,1,1,1
Accident_Severity,3,3,3,2,2
Number_of_Vehicles,2,1,2,2,2
Number_of_Casualties,2,1,1,1,2
Date,01/01/2018,01/01/2018,01/01/2018,01/01/2018,01/01/2018


In [5]:
df_accidents_regress = pd.DataFrame(accidents_s1, columns=['Number_of_Vehicles', 'Day_of_Week',
       'Time', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Fatal', 'Serious', 'Slight'])

df_accidents_regress["Time"] = pd.to_datetime(df_accidents_regress["Time"], format= '%H:%M').dt.time

In [6]:
df_accidents_regress_nonan = df_accidents_regress.dropna()

In [7]:
# Label encoder
from sklearn.preprocessing import LabelEncoder
lblE = LabelEncoder()
for i in df_accidents_regress_nonan:
    if df_accidents_regress_nonan[i].dtype == 'object':
        #print(df_accidents_regress[i])
        lblE.fit(df_accidents_regress_nonan[i])
        df_accidents_regress_nonan[i] = lblE.transform(df_accidents_regress_nonan[i])

C:\Users\Gabor.Jenei\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
X = df_accidents_regress_nonan[['Number_of_Vehicles', 'Day_of_Week',
       'Time', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area']]
y =  df_accidents_regress_nonan[['Fatal', 'Serious', 'Slight']]

In [10]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

X[['Number_of_Vehicles', 'Day_of_Week',
       'Time', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area']] = scale.fit_transform(X[['Number_of_Vehicles', 'Day_of_Week',
       'Time', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area']].as_matrix())
y[['Fatal', 'Serious', 'Slight']] = scale.fit_transform(y[['Fatal', 'Serious', 'Slight']].as_matrix())

C:\Users\Gabor.Jenei\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Gabor.Jenei\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\Gabor.Jenei\AppData\Roaming\Python\Python36\sit

In [11]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestRegressor

m = RandomForestRegressor(n_estimators=100)
m.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [13]:
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))
print(np.sqrt(((m.predict(X_train)-y_train)**2).mean()))
print(np.sqrt(((m.predict(X_test)-y_test)**2).mean()))

0.8234208835614866
-0.06841623569153335
Fatal      0.412694
Serious    0.430762
Slight     0.417876
dtype: float64
Fatal      1.046817
Serious    1.045562
Slight     0.998963
dtype: float64


In [14]:
f_imp = pd.DataFrame(data={'importance':m.feature_importances_,'features':X_train.columns}).set_index('features')
f_imp = f_imp.sort_values('importance', ascending=False)
f_imp.head(20)

,importance
features,
Time,0.334683
1st_Road_Number,0.231634
Day_of_Week,0.095398
2nd_Road_Number,0.041129
Weather_Conditions,0.031245
1st_Road_Class,0.030127
Number_of_Vehicles,0.028739
Junction_Detail,0.027543
Speed_limit,0.027028


### Linear Regression

In [15]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split
reg = linear_model.LinearRegression()
clf = reg.fit(X_train, y_train)

In [16]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print(np.sqrt(((clf.predict(X_train)-y_train)**2).mean()))
print(np.sqrt(((clf.predict(X_test)-y_test)**2).mean()))

0.03131507523502639
0.03169997746751671
Fatal      0.995954
Serious    0.990857
Slight     0.967718
dtype: float64
Fatal      0.991187
Serious    0.997856
Slight     0.954017
dtype: float64


### SDG

In [17]:
sdg = linear_model.SGDRegressor()

In [18]:
sdg.fit(X_train, y_train["Fatal"])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [19]:
print(sdg.score(X_train, y_train["Fatal"]))
print(sdg.score(X_test, y_test["Fatal"]))
print(np.sqrt(((sdg.predict(X_train)-y_train["Fatal"])**2).mean()))
print(np.sqrt(((sdg.predict(X_test)-y_test["Fatal"])**2).mean()))

0.008751864573156576
0.008811927551068854
0.9965585387474526
0.9917976834674034


In [ ]:
scores = cross_val_score(m, X, y, cv=5)
print(scores)
print(scores.mean())

In [21]:
scores = cross_val_score(clf, X, y, cv=5)
print(scores)
print(scores.mean())

[0.02241142 0.02902346 0.03323155 0.03366478 0.03104239]
0.029874720019560604


In [ ]:
scores = cross_val_score(sdg, X, y, cv=5)
print(scores)
print(scores.mean())